In [1]:
DATASET_PATH="/data/edu/projects/learn/ml/deeplearning/data/caltech101"

### Split the dataset into training, validation and test sets

In [3]:
def copy_files(src, dst):
    import os, shutil
    
    if not os.path.exists(dst):
        os.makedirs(dst)
    
    for file in src:
        shutil.copy2(file, dst)

def split_data(dataset, ratio=(0.6,0.2,0.2)):
    import os, shutil
    from random import shuffle
    
    data = {}
    data['labels'] = [x for x in os.listdir(os.path.join(dataset,'raw')) if os.path.isdir(os.path.join(dataset, 'raw', x))]
    data['training'], data['validation'], data['test'] = {}, {}, {}
    
    for label in data['labels']:
        images = [x for x in os.listdir(os.path.join(dataset, 'raw', label))]
        
        shuffle(images)
        
        train_idx = int(len(images) * ratio[0])
        val_idx = train_idx + int(len(images) * ratio[1])
        training, validation, test = images[slice(0,train_idx)], images[slice(train_idx,val_idx)], images[slice(val_idx,len(images))]
        
        data['training'][label] = [os.path.join(dataset,'raw',label,image) for image in training]
        data['validation'][label] = [os.path.join(dataset,'raw',label,image) for image in validation]
        data['test'][label] = [os.path.join(dataset,'raw',label,image) for image in test]
        
        copy_files(data['training'][label], os.path.join(dataset, 'run', 'training', label))
        copy_files(data['validation'][label], os.path.join(dataset, 'run', 'validation', label))
        copy_files(data['test'][label], os.path.join(dataset, 'run', 'test', label))
    
    return data

data = split_data(DATASET_PATH)

### Create One Hot encoding for labels

In [4]:
from sklearn.preprocessing import LabelBinarizer
labels_onehot = LabelBinarizer().fit_transform(data['labels'])

### Setup Preprocessing for images

In [5]:
import os
import keras
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

train_idg = ImageDataGenerator(
      samplewise_center            = True
    , samplewise_std_normalization = True
    , fill_mode                    = 'nearest'
    , rescale                      = 1./255
)

train_idg_generator = train_idg.flow_from_directory(
      directory   = os.path.join(DATASET_PATH,'run','training')
    , target_size = (224,224)
    , color_mode  = 'rgb'
    , class_mode  = 'categorical'
    , batch_size  = 32
)

val_idg = ImageDataGenerator(
      rescale                      = 1./255
)

val_idg_generator = val_idg.flow_from_directory(
      directory   = os.path.join(DATASET_PATH,'run','validation')
    , target_size = ()
    , color_mode  = 'rgb'
    , class_mode  = 'categorical'
)

Using TensorFlow backend.


Found 5171 images belonging to 101 classes.
Found 1695 images belonging to 101 classes.


### Build VGG19 Network

In [6]:
from keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense, Dropout
from keras.initializers import he_normal

vgg19 = [
    
    ## Block 1
      Conv2D(filters=64, kernel_size=(3,3), padding='same', kernel_regularizer=keras.regularizers.l2(0.01), name='conv1_1', input_shape=(224,224,3))
    , BatchNormalization()
    , Activation('relu')
    , Conv2D(filters=64, kernel_size=(3,3), padding='same', kernel_initializer=he_normal(), name='conv1_2')
    , BatchNormalization()
    , Activation('relu')
    , MaxPooling2D((2,2), strides=(2,2), name='conv1_pool')
    
    ## Block 2
    , Conv2D(filters=128, kernel_size=(3,3), padding='same', kernel_initializer=he_normal(), name='conv2_1')
    , BatchNormalization()
    , Activation('relu')
    , Conv2D(filters=128, kernel_size=(3,3), padding='same', kernel_initializer=he_normal(), name='conv2_2')
    , BatchNormalization()
    , Activation('relu')
    , MaxPooling2D((2,2), strides=(2,2), name='conv2_pool')
    
    ## Block 3
    , Conv2D(filters=256, kernel_size=(3,3), padding='same', kernel_initializer=he_normal(), name='conv3_1')
    , BatchNormalization()
    , Activation('relu')
    , Conv2D(filters=256, kernel_size=(3,3), padding='same', kernel_initializer=he_normal(), name='conv3_2')
    , BatchNormalization()
    , Activation('relu')
    , Conv2D(filters=256, kernel_size=(3,3), padding='same', kernel_initializer=he_normal(), name='conv3_3')
    , BatchNormalization()
    , Activation('relu')
    , Conv2D(filters=256, kernel_size=(3,3), padding='same', kernel_initializer=he_normal(), name='conv3_4')
    , BatchNormalization()
    , Activation('relu')
    , MaxPooling2D((2,2), strides=(2,2), name='conv3_pool')
    
    ## Block 4
    , Conv2D(filters=512, kernel_size=(3,3), padding='same', kernel_initializer=he_normal(), name='conv4_1')
    , BatchNormalization()
    , Activation('relu')
    , Conv2D(filters=512, kernel_size=(3,3), padding='same', kernel_initializer=he_normal(), name='conv4_2')
    , BatchNormalization()
    , Activation('relu')
    , Conv2D(filters=512, kernel_size=(3,3), padding='same', kernel_initializer=he_normal(), name='conv4_3')
    , BatchNormalization()
    , Activation('relu')
    , Conv2D(filters=512, kernel_size=(3,3), padding='same', kernel_initializer=he_normal(), name='conv4_4')
    , BatchNormalization()
    , Activation('relu')
    , MaxPooling2D((2,2), strides=(2,2), name='conv4_pool')
    
    ## Block 5
    , Conv2D(filters=1024, kernel_size=(3,3), padding='same', kernel_initializer=he_normal(), name='conv5_1')
    , BatchNormalization()
    , Activation('relu')
    , Conv2D(filters=1024, kernel_size=(3,3), padding='same', kernel_initializer=he_normal(), name='conv5_2')
    , BatchNormalization()
    , Activation('relu')
    , Conv2D(filters=1024, kernel_size=(3,3), padding='same', kernel_initializer=he_normal(), name='conv5_3')
    , BatchNormalization()
    , Activation('relu')
    , Conv2D(filters=1024, kernel_size=(3,3), padding='same', kernel_initializer=he_normal(), name='conv5_4')
    , BatchNormalization()
    , Activation('relu')
    , MaxPooling2D((2,2), strides=(2,2), name='conv5_pool')
    
    ## Block 6
    , Flatten(name='flatten')
    , Dense(units=4096, use_bias=True, kernel_initializer=he_normal(), name='fc1')
    , BatchNormalization()
    , Activation('relu')
    , Dropout(0.4)
    , Dense(units=4096, use_bias=True, kernel_initializer=he_normal(), name='fc2')
    , BatchNormalization()
    , Activation('relu')
    , Dropout(0.4)
    , Dense(units=101, kernel_initializer=he_normal(), name='predictions')
    , BatchNormalization()
    , Activation('softmax')
]

model = Sequential(vgg19)

### Compile the mode

In [7]:
from keras.optimizers import Adam

model.compile(
      optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=1e-6)
    , loss      = 'categorical_crossentropy'
    , metrics   = ['acc']
)

### Train the model

In [8]:
from keras.callbacks import TensorBoard

model.fit_generator(
      generator        = train_idg_generator
    , validation_data  = val_idg_generator
    , epochs           = 50
    , steps_per_epoch  = 5171/32
    , validation_steps = 1695/32
    , max_queue_size   = 100
    , callbacks        = [TensorBoard(
          log_dir='/data/tmp/trash/caltech101'
    )]
)

ResourceExhaustedError: OOM when allocating tensor with shape[50176,4096] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node training/Adam/Variable_64/Assign (defined at /opt/anaconda2/envs/tensorflow/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:402)  = Assign[T=DT_FLOAT, _grappler_relax_allocator_constraints=true, use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training/Adam/Variable_64, training/Adam/zeros_140)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'training/Adam/Variable_64/Assign', defined at:
  File "/opt/anaconda2/envs/tensorflow/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/anaconda2/envs/tensorflow/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/anaco